# Exploring the extracted ROI timeseries



In [2]:
import numpy as np
import nibabel as nib
import glob
import matplotlib.pyplot as plt
from brainiak.eventseg import event
from scipy.stats import zscore
import matplotlib.patches as patches

%autosave 5


Autosaving every 5 seconds


In [3]:
all_roi_timeseries = glob.glob('/Users/jahchwe/neurohack/nh-paranoia/schaeffer_dict_roi/*')
all_roi_timeseries

[]

In [5]:
all_roi_timeseries = glob.glob('/home/jovyan/paranoia_project/schaeffer_dict_roi/*')


In [6]:
# sanity check that each timeseries has none zero 
# reformat data into a dictionary with subject and ROI keys, with timeseries as entries

all_data = {}

for f in all_roi_timeseries[68:70]:
    
    idx = f.split('/')[-1]
    idx = idx.split('_')[:-2]
    idx = '_'.join(idx)
    
    data = np.load(f, allow_pickle = True).item()
    for d in data:
        #print(data[d].shape)
        None
    all_data[idx] = data
    
# voxels x TRs
# one row per subject per ROI

In [12]:
all_roi_timeseries[68:70]

['/home/jovyan/paranoia_project/schaeffer_dict_roi/7Networks_RH_SomMot_7_all_subs.npy',
 '/home/jovyan/paranoia_project/schaeffer_dict_roi/7Networks_RH_DorsAttn_Post_1_all_subs.npy']

# Visualizing the HMM for all ROIs
## create functions for iteration over all ROIs

In [7]:
# roi data in the form of subjects X voxels X TRs 

def run_hmm(roi_data):
    
    k_array = np.arange(2,50,5)
    test_ll = np.zeros(len(k_array))
    nsubj = roi_data.shape[0]
    print(nsubj)

    # note that the HMM takes a time x voxel ndarray. 

    # Loop over possible settings for the number of events k
    print('Trying events:', end=' ')
    for i, k in enumerate(k_array):
        print(k, end=' ')

        # Fit HMM to half of the subjects
        train = zscore(roi_data[:(nsubj//2),:,:].mean(0), axis=1).T
        HMM = event.EventSegment(k)
        HMM.fit(train)

        # Test log-likelihood (model fit) on other half of subjects
        test = zscore(roi_data[(nsubj//2):,:,:].mean(0), axis=1).T
        _, test_ll[i] = HMM.find_events(test)

    best_k = k_array[np.argmax(test_ll)]
    print('\nOptimal # of events: ', best_k)


    # Do a final fit with the optimal number of events, using all subjects
    # Set split_merge=True to prioritize fit quality over speed
    es = event.EventSegment(best_k, split_merge=True)
    es.fit(zscore(roi_data.mean(0), axis=1).T)
    return es

In [16]:
def plot_correlation_w_boundaries(da_list):
    # list of lists, where inner entries are [roi, hmm_fitted]
    # Plot timepoint-timepoint correlation matrix
    
    # da list [0] = roi
    # da list [1] = roi_data
    # da list [2] = eb
        
    for it in range(len(da_list)):
        fig, ax = plt.subplots()
        ax.imshow(np.corrcoef(zscore(da_list[it][1].mean(0), axis=1).T), cmap = 'viridis')
        ax.set_title(da_list[it][0])
        ax.set_xlabel('Timepoint')
        ax.set_ylabel('Timepoint')

        # Plot boundaries as boxes on top of correlation matrix
        bounds_aug = np.concatenate(([0], da_list[it][2], [da_list[it][1].shape[1]]))
        for i in range(len(bounds_aug) - 1):
            rect = patches.Rectangle(
                (bounds_aug[i], bounds_aug[i]),
                bounds_aug[i+1] - bounds_aug[i],
                bounds_aug[i+1] - bounds_aug[i],
                linewidth = 2, edgecolor = 'w',facecolor = 'none'
            )
            ax.add_patch(rect)
        
        #plt.savefig('figs/%s_corr_w_boundaries.jpg' % da_list[it][0], dpi = 200)
    

In [13]:
hmm_boundaries = []

for roi in all_data.keys():
    roi_data = np.array([all_data[roi][x] for x in all_data[roi].keys()])
    #assert roi_data.shape[0] == 14 and roi_data.shape[2] == 526
    
    try:
        hmm_fit = run_hmm(roi_data)

        eb = np.where(np.diff(np.argmax(hmm_fit.segments_[0],axis=1)))[0]
        hmm_boundaries.append([roi, roi_data, eb]) 
        
    except:
        continue

17
Trying events: 2 17
Trying events: 2 

In [17]:
plot_correlation_w_boundaries(hmm_boundaries)